<a href="https://colab.research.google.com/github/mottasilvia/UCU-NLP/blob/main/Proyecto%20Final/PLN_Proyecto_Final_BETO_con_datos_preprocesados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo BETO CASED con datos preprocesados y clases agrupadas y combinadas

| **Clase**                | **Precision** | **Recall** | **F1-score** | **Support** |
|--------------------------|---------------|------------|--------------|-------------|
| agradecimiento o saludo  | 0.94          | 0.96       | 0.95         | 463         |
| autorespuesta            | 1.00          | 0.90       | 0.95         | 21          |
| baja                     | 0.88          | 0.79       | 0.83         | 19          |
| equivocado               | 0.87          | 0.81       | 0.84         | 16          |
| glicemia                 | 0.62          | 0.29       | 0.40         | 17          |
| medicación o turno       | 0.78          | 0.68       | 0.72         | 37          |
| optin                    | 0.98          | 0.90       | 0.94         | 383         |
| otros                    | 0.74          | 0.78       | 0.76         | 51          |
| respuesta                | 0.84          | 0.92       | 0.88         | 395         |
| **accuracy**             |               |            | 0.91         | 1402        |
| **macro avg**            | 0.85          | 0.78       | 0.81         | 1402        |
| **weighted avg**         | 0.91          | 0.91       | 0.91         | 1402        |


In [ ]:
!pip install transformers pandas scikit-learn accelerate emoji



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 24.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive
import emoji

In [ ]:
# Verificar si hay una GPU disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Imprimir información sobre los dispositivos CUDA
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


Usando dispositivo: cuda
True
1
Tesla T4


In [ ]:
_# ejecutar luego de subir archivo excel o tener vinculado Google Drive
import os
import pandas as pd
from google.colab import drive

if os.path.isfile("/content/mensajes_trabajo_final.xlsx"):
  data = pd.read_excel("/content/mensajes_trabajo_final.xlsx")
else:
  # Montar Google Drive
  drive.mount('/content/drive')

  # Cambia el path al archivo que hayas subido a tu Google Drive
  file_path = '/content/drive/MyDrive/UCU-NLP/mensajes_preprocesados.csv'

  # Cargar el archivo Excel
  data = pd.read_csv(file_path)

print("Datos cargados")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Datos cargados


In [ ]:
# Cargar el archivo csv
data = pd.read_csv(file_path)


In [ ]:
# Seleccionar las columnas relevantes
data = data[['enviado','recibido', 'sin_stop_words_significado_emoticones','clase']]

# Mostrar las primeras filas del archivo
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido  \
0                                               👍🏽🫶🏽   
1                                           Hola....   
2                                       👏🏻👏🏻 gracias   
3                                            Gracias   
4  Hola .. si suele..cuando me...desvelo..o de ta...   

  sin_stop_words_significado_emoticones                           clase  
0                     aprobación afecto                  agradecimiento  
1                                  hola                          saludo  
2             aplausos aplausos gracias                  agradecimiento  
3                             

In [ ]:
# unir enviado con recibido
#data['recibido_original'] = data['recibido']
data['recibido'] = data['enviado'] + " | " + data['sin_stop_words_significado_emoticones'].astype(str)
#data['recibido'] = data['enviado'] + " | " + data['recibido'].astype(str)

In [ ]:
# Seleccionar las columnas relevantes
data = data[[ 'recibido',  'clase']]

In [ ]:
print(data.head())

                                             enviado  \
0   ¡Limpiar la casa, subir escaleras, trabajar e...   
1   ¡Limpiar la casa, subir escaleras, trabajar e...   
2   ¡Limpiar la casa, subir escaleras, trabajar e...   
3   ¡Limpiar la casa, subir escaleras, trabajar e...   
4   ¡Limpiar la casa, subir escaleras, trabajar e...   

                                            recibido                    clase  
0   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo  
1   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo  
2   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo  
3   ¡Limpiar la casa, subir escaleras, trabajar e...  agradecimiento o saludo  
4   ¡Limpiar la casa, subir escaleras, trabajar e...                respuesta  


In [ ]:
# agrupar clases

data.loc[data['clase'] == 'agradecimiento', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'saludo', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'medicación', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'turno', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'queja', 'clase'] = 'otros'
data.loc[data['clase'] == 'pregunta', 'clase'] = 'otros'
data.loc[data['clase'] == 'respuesta a consejo o pregunta', 'clase'] = 'respuesta'

In [ ]:
# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

In [ ]:
data.head()

,recibido,clase,clases_cat
0,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
1,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
2,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
3,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
4,"¡Limpiar la casa, subir escaleras, trabajar e...",respuesta,8


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU está disponible y se usará.")
else:
  device = torch.device("cpu")
  print("GPU no está disponible, se usará la CPU.")


GPU está disponible y se usará.


In [ ]:
## Aplicar la conversión de emoticones a la columna 'recibido'
#data['recibido'] = data['recibido'].apply(convertir_emoticones)

# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
#data['clase'] = data['clase'].astype('category')
#data['clases_cat'] = data['clase'].cat.codes

# Filtrar las filas con etiquetas inválidas
data = data[data['clases_cat'] >= 0]

# Verificar que todas las etiquetas estén en el rango correcto
num_labels = len(data['clase'].unique())
print(f"Numero de etiquetas: {num_labels}")
print(f"Etiquetas unicas: {data['clases_cat'].unique()}")
assert data['clases_cat'].min() >= 0 and data['clases_cat'].max() < num_labels

Numero de etiquetas: 9
Etiquetas unicas: [0 8 2 3 7 6 5 1 4]


In [ ]:
# Seleccionar las columnas relevantes
data = data[[ 'recibido',  'clase','clases_cat']]

In [ ]:
data.head()

,recibido,clase,clases_cat
0,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
1,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
2,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
3,"¡Limpiar la casa, subir escaleras, trabajar e...",agradecimiento o saludo,0
4,"¡Limpiar la casa, subir escaleras, trabajar e...",respuesta,8


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


print(f'Tamaño del conjunto de entrenamiento: {len(train_data)}')
print(f'Tamaño del conjunto de prueba: {len(test_data)}')

Tamaño del conjunto de entrenamiento: 5606
Tamaño del conjunto de prueba: 1402


In [ ]:
# Crear el tokenizador
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [ ]:
# Función para tokenizar y convertir en tensores
def tokenize_data(data):
    # Asegurar que la columna 'recibido' contenga solo cadenas
    data['recibido'] = data['recibido'].astype(str)

    # Tokenizar los datos
    inputs = tokenizer(data['recibido'].tolist(), padding=True, truncation=True, return_tensors="pt")

    # Convertir etiquetas de clase a tensores
    labels = torch.tensor(data['clases_cat'].tolist())
    inputs['labels'] = labels

    return inputs


In [ ]:
# Tokenizar los datos de entrenamiento y prueba
train_inputs = tokenize_data(train_data)
test_inputs = tokenize_data(test_data)

# Calcular los pesos de las clases en la CPU
class_weights = compute_class_weight(class_weight='balanced', classes=data['clases_cat'].unique(), y=train_data['clases_cat'])
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [ ]:
train_data.head()

,recibido,clase,clases_cat
1756,La DIABETES es una enfermedad que dura toda l...,otros,7
3138,Tomar medicamentos para la DIABETES evita que...,respuesta,8
1885,La insulina evita que la DIABETES avance. Ayu...,agradecimiento o saludo,0
4676,"¿Sabías que los bizcochos, las galletitas, y l...",respuesta,8
390,¿Sabías que la DIABETES es una enfermedad que...,respuesta,8


In [ ]:
# Definir un Dataset personalizado
class CustomDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        # Removed the .to(device) call as tensors will be moved to device in the Trainer
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

train_dataset = CustomDataset(train_inputs)
test_dataset = CustomDataset(test_inputs)

In [ ]:


# Crear el modelo
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=num_labels)
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
 # Crear una función de pérdida personalizada para incluir los pesos de las clases
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(device)
        inputs = {k: v.to(device) for k, v in inputs.items() if k != "labels"}
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Crear el objeto Trainer con la función de pérdida personalizada
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Cambiado a evaluation_strategy
    per_device_train_batch_size=4,  # Reducir el tamaño del lote para evitar problemas de memoria
    per_device_eval_batch_size=4,   # Reducir el tamaño del lote para evaluación también
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.539300,0.481005
2,0.343700,0.448778
3,0.139000,0.489872


TrainOutput(global_step=4206, training_loss=0.3680028176228319, metrics={'train_runtime': 1106.0123, 'train_samples_per_second': 15.206, 'train_steps_per_second': 3.803, 'total_flos': 1953346179895704.0, 'train_loss': 0.3680028176228319, 'epoch': 3.0})

In [ ]:
# Evaluar el modelo
eval_results = trainer.evaluate()
print(f"Resultados de la evaluación: {eval_results}")

# Evaluación detallada con classification_report y confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

# Predecir etiquetas para el conjunto de prueba
y_pred = trainer.predict(test_dataset).predictions.argmax(axis=1)
y_true = test_inputs['labels'].numpy()

# Imprimir el reporte de clasificación
report = classification_report(y_true, y_pred, target_names=data['clase'].cat.categories)
print(report)

# Imprimir la matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print(cm)

Resultados de la evaluación: {'eval_loss': 0.4898715019226074, 'eval_runtime': 15.1043, 'eval_samples_per_second': 92.821, 'eval_steps_per_second': 23.238, 'epoch': 3.0}
                         precision    recall  f1-score   support

agradecimiento o saludo       0.97      0.97      0.97       463
          autorespuesta       1.00      0.95      0.98        21
                   baja       0.94      0.84      0.89        19
             equivocado       0.81      0.81      0.81        16
               glicemia       0.93      0.76      0.84        17
     medicación o turno       0.72      0.76      0.74        37
                  optin       0.99      0.98      0.99       383
                  otros       0.65      0.61      0.63        51
              respuesta       0.89      0.91      0.90       395

               accuracy                           0.93      1402
              macro avg       0.88      0.84      0.86      1402
           weighted avg       0.93      0.93    

| **Clase**                | **Precision** | **Recall** | **F1-score** | **Support** |
|--------------------------|---------------|------------|--------------|-------------|
| agradecimiento o saludo  | 0.94          | 0.96       | 0.95         | 463         |
| autorespuesta            | 1.00          | 0.90       | 0.95         | 21          |
| baja                     | 0.88          | 0.79       | 0.83         | 19          |
| equivocado               | 0.87          | 0.81       | 0.84         | 16          |
| glicemia                 | 0.62          | 0.29       | 0.40         | 17          |
| medicación o turno       | 0.78          | 0.68       | 0.72         | 37          |
| optin                    | 0.98          | 0.90       | 0.94         | 383         |
| otros                    | 0.74          | 0.78       | 0.76         | 51          |
| respuesta                | 0.84          | 0.92       | 0.88         | 395         |
| **accuracy**             |               |            | 0.91         | 1402        |
| **macro avg**            | 0.85          | 0.78       | 0.81         | 1402        |
| **weighted avg**         | 0.91          | 0.91       | 0.91         | 1402        |


In [ ]:
# Mostrar y_true y y_pred
print("y_true:", y_true)
print("y_pred:", y_pred)

y_true: [6 0 0 ... 6 8 5]
y_pred: [6 0 0 ... 6 8 5]


In [ ]:
# Mostrar y_true, y_pred y los datos originales
results = pd.DataFrame({
    'Mensaje Original': test_data['recibido'],
    'Etiqueta Real': y_true,
    'Predicción': y_pred,
    'Clase Real': test_data['clase'].cat.categories[y_true],
    'Clase Predicha': test_data['clase'].cat.categories[y_pred]
})

print(results)

                                       Mensaje Original  Etiqueta Real  \
5153  El Programa de Diabetes nunca te va a pedir un...              6   
3607  ¡Te damos la bienvenida! Recibirás un mensaje ...              0   
246    ¿Qué podés hacer hoy para acercarte a cumplir...              0   
3291  ¡Te damos la bienvenida! Recibirás un mensaje ...              6   
5843  Necesitamos tu aprobación para enviarte mensaj...              3   
...                                                 ...            ...   
3657  ¡Te damos la bienvenida! Recibirás un mensaje ...              6   
4180  ¡Te damos la bienvenida! Recibirás un mensaje ...              6   
6929  Te damos la bienvenida al programa de prevenci...              6   
1335   En la visita con el médico, habla los temas q...              8   
5215  En la visita con el médico, habla los temas qu...              5   

      Predicción               Clase Real           Clase Predicha  
5153           6                    optin 

In [ ]:
# Guardar el DataFrame en un archivo CSV
if os.path.isfile("/content/mensajes_trabajo_final.xlsx"):
  results.to_csv('/content/resultados_detallados.csv', index=False)
  print("Resultados guardados en '/content/drive/MyDrive/UCU-NLP/resultados_detallados_combinados_preprocesados.csv'")
else:
  results.to_csv('/content/drive/MyDrive/UCU-NLP/resultados_detallados_combinados_preprocesados.csv', index=False)
  print("Resultados guardados en '/content/drive/MyDrive/UCU-NLP/resultados_detallados_combinados_preprocesados.csv'")


Resultados guardados en '/content/drive/MyDrive/UCU-NLP/resultados_detallados_combinados_preprocesados.csv'


#
################################################################################################################################################################